# Record Embedding

In [5]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

## Embedding using fastText
Details here: https://fasttext.cc/

In [8]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import FastText

Convert each tuple into a row

In [39]:
#hospital dataset

#1. load hospital dataset and turthvalue.csv; mark error for unmatched values and cleanup using FastText; fill empty cells with word2vec
#finding errors and outliers -> discover FDs and try marking errors? or word2vec most_similar of val and if row does not contain the cell values??!

df_hospital = pd.read_csv('dirty_data_transformed.csv', dtype=object)
df_truth = pd.read_csv('truthvalue.csv', dtype=object)



provider = df_truth['ProviderNumber'].tolist()
hospital = df_truth['HospitalName'].tolist()
address = df_truth['Address1'].tolist()
city = df_truth['City'].tolist()
state = df_truth['State'].tolist()
zipcode = df_truth['ZipCode'].tolist()
county = df_truth['CountyName'].tolist()
phone = df_truth['PhoneNumber'].tolist()
hospType = df_truth['HospitalType'].tolist()
owner = df_truth['HospitalOwner'].tolist()
service = df_truth['EmergencyService'].tolist()
condition = df_truth['Condition'].tolist()
code = df_truth['MeasureCode'].tolist()
name = df_truth['MeasureName'].tolist()
score = df_truth['Score'].tolist()
sample = df_truth['Sample'].tolist()
stateavg = df_truth['Stateavg'].tolist()

combined_hosp = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))



In [40]:
len(df_truth)

1000

In [24]:
len(combined_hosp[1])

17

In [41]:
#hospital dirty dataset; maintain dirty dataset separately
provider = df_hospital['ProviderNumber'].tolist()
hospital = df_hospital['HospitalName'].tolist()
address = df_hospital['Address1'].tolist()
city = df_hospital['City'].tolist()
state = df_hospital['State'].tolist()
zipcode = df_hospital['ZipCode'].tolist()
county = df_hospital['CountyName'].tolist()
phone = df_hospital['PhoneNumber'].tolist()
hospType = df_hospital['HospitalType'].tolist()
owner = df_hospital['HospitalOwner'].tolist()
service = df_hospital['EmergencyService'].tolist()
condition = df_hospital['Condition'].tolist()
code = df_hospital['MeasureCode'].tolist()
name = df_hospital['MeasureName'].tolist()
score = df_hospital['Score'].tolist()
sample = df_hospital['Sample'].tolist()
stateavg = df_hospital['Stateavg'].tolist()

combined_dirty = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))
len(combined_dirty)




1000

In [42]:
len(combined_dirty)

1000

In [48]:
print (dirty_list[10],truth_list[10])


Hearx Axxack Heart Attack


In [45]:
#training the model with truth data.
model_hosp = FastText(combined_hosp, min_count=1, workers=8, iter=100)

In [65]:
len(dirty_list)

594

In [49]:
#result is a list of tuples of (predicted_value, dirty_value, truth_value)
result = []
for i in range(len(dirty_list)):
    if isinstance(dirty_list[i], str):
        try:
            result.append((model_hosp.most_similar(dirty_list[i])[0][0],dirty_list[i],truth_list[i]))
        except KeyError:
            #print(dirty_list[i])
            pass

        
    

In [57]:
#identify the true positives and false positives. Store them in the seperate list for further analysis.
true_pos = 0
false_pos = 0
correctly_predicted = []
falsely_predicted = []
for i in range(len(result)):
    if result[i][0] == result[i][2]:
        true_pos = true_pos + 1
        correctly_predicted.append(result[i])
    else:
        false_pos = false_pos + 1
        falsely_predicted.append(result[i])




367 176


In [67]:
correctly_predicted[2]

('surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream �C not a razor)',
 'surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream C not a razor)',
 'surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream �C not a razor)')

In [68]:
falsely_predicted[2]

('BOAZ', 'x5957', '35957')

In [69]:
model_hosp.most_similar('x5957')

[('BOAZ', 0.9628452062606812),
 ('MARSHALL', 0.931229829788208),
 ('35957', 0.8796513080596924),
 ('2565938310', 0.8265697956085205),
 ('2505 U S HIGHWAY 431 NORTH', 0.8258407115936279),
 ('35976', 0.785955548286438),
 ('2565718000', 0.6646440625190735),
 ('MARSHALL MEDICAL CENTER SOUTH', 0.6296823620796204),
 ('MARSHALL MEDICAL CENTER NORTH', 0.6226513385772705),
 ('15155 HIGHWAY 43', 0.6163424253463745)]